In [23]:
# %% [markdown]
# # Benchmark XGBoost & Ollama – CPU vs GPU per macchina
#
# Requisiti:
#   pip install pandas altair==5.5 vega_datasets

# %%
import pandas as pd
import altair as alt
from pathlib import Path
import os

alt.data_transformers.disable_max_rows()

# %% ------------ Lettura CSV e feature engineering ------------
xgb_df    = pd.read_csv(Path("results/xgb_reference.csv"))
ollama_df = pd.read_csv(Path("results/ollama_reference.csv"))

if os.path.exists("results/xgb.csv"):
    xgb_df = pd.concat([xgb_df, pd.read_csv("results/xgb.csv")], ignore_index=True)

if os.path.exists("results/ollama.csv"):
    ollama_df = pd.concat([ollama_df, pd.read_csv("results/ollama.csv")], ignore_index=True)

for df in (xgb_df, ollama_df):
    df["machine_label"] = df["CPU"] + " + " + df["GPU"]

    # Combina macchina + CPU/GPU (8 combinazioni attese: 4 macchine × 2)
    df["machine_gpu"] = (
        df["machine_label"] + " — " + df["gpu"].map({True: "GPU", False: "CPU"})
    )

# Ordine naturale delle dimensioni dataset
xgb_df["dataset_rows"] = xgb_df["dataset_rows"].astype(str)
row_order = sorted(
    xgb_df["dataset_rows"].unique(),
    key=lambda v: float("inf") if v == "full" else int(v)
)
xgb_df["dataset_rows"] = pd.Categorical(xgb_df["dataset_rows"],
                                        categories=row_order,
                                        ordered=True)

# %% ------------------- Parametri (menu tendevoli) --------------
cpu_param  = alt.param("cpu",
    bind=alt.binding_select(options=["All"] + sorted(xgb_df["CPU"].unique()),
                            name="CPU: "),
    value="All"
)
gpu_param  = alt.param("gpu",
    bind=alt.binding_select(options=["All"] + sorted(xgb_df["GPU"].unique()),
                            name="GPU modello: "),
    value="All"
)
mach_param = alt.param("mach",
    bind=alt.binding_select(options=["All"] + sorted(xgb_df["machine_label"].unique()),
                            name="Macchina: "),
    value="All"
)
rows_param = alt.param("rows",
    bind=alt.binding_select(options=["All"] + row_order, name="Rows: "),
    value="All"
)
mdl_param  = alt.param("mdl",
    bind=alt.binding_select(options=["All"] + sorted(ollama_df["model"].unique()),
                            name="Modello: "),
    value="All"
)
gpu_tf_param = alt.param("gpu_tf",
    bind=alt.binding_select(options=["All", "GPU", "CPU"], name="GPU used: "),
    value="All"
)

# leggenda cliccabile per multi‑selezione sulle macchine
mach_legend_sel = alt.selection_point(fields=["machine_label"], bind="legend")

# %% ------------------- Espressioni di filtro --------------------
cpu_filter  = "(cpu == 'All')  || (datum.CPU  == cpu)"
gpu_filter  = "(gpu == 'All')  || (datum.GPU  == gpu)"
mach_filter = "(mach == 'All') || (datum.machine_label == mach)"
rows_filter = "(rows == 'All') || (datum.dataset_rows == rows)"
mdl_filter  = "(mdl == 'All')  || (datum.model == mdl)"
gpu_tf_filter = (
    "(gpu_tf == 'All')"
    " || ((gpu_tf == 'GPU') && datum.gpu)"
    " || ((gpu_tf == 'CPU') && !datum.gpu)"
)

opacity_scale = alt.Scale(domain=[False, True], range=[1.0, 0.65])

# %% ---------------- Grafico XGBoost -----------------------------
base_xgb = (
    alt.Chart(xgb_df)
    .transform_filter(cpu_filter)
    .transform_filter(gpu_filter)
    .transform_filter(mach_filter)
    .transform_filter(rows_filter)
    .transform_filter(gpu_tf_filter)
    .transform_filter(mach_legend_sel)
)

bars_xgb = (
    base_xgb.mark_bar(orient="horizontal", stroke='black')
    .encode(
        y=alt.Y("dataset_rows:N", sort=row_order,
                title="Numero di righe del dataset"),
        yOffset="machine_gpu:N",                # 8 combinazioni
        x=alt.X("train_median_s:Q", title="Train time [s]"),
        color=alt.Color("machine_label:N",
                        legend=alt.Legend(title="Macchina")),
        opacity=alt.Opacity("gpu:N",
                            scale=opacity_scale, legend=None),
        strokeWidth=alt.condition(alt.datum.reference, alt.value(0.5), alt.value(3)),
        tooltip=[
            "CPU", "GPU", "machine_gpu", "dataset_rows", "gpu",
            alt.Tooltip("train_median_s:Q", title="Training time [s]")
        ]
    )
)

text_xgb = bars_xgb.mark_text(align="left", dx=6).encode(
    text=alt.Text("train_median_s:Q", format=".2f")
)

chart_xgb = (bars_xgb + text_xgb).properties(
    width=650,
    title="Tempo di training – CPU vs GPU, per macchina"
).add_params(
    cpu_param, gpu_param, mach_param, rows_param,
    gpu_tf_param, mach_legend_sel
)

# %% ---------------- Grafico Ollama ------------------------------
base_oll = (
    alt.Chart(ollama_df)
    .transform_filter(cpu_filter)
    .transform_filter(gpu_filter)
    .transform_filter(mach_filter)
    .transform_filter(mdl_filter)
    .transform_filter(gpu_tf_filter)
    .transform_filter(mach_legend_sel)
)

bars_oll = (
    base_oll.mark_bar(orient="horizontal", stroke='black')
    .encode(
        y=alt.Y("model:N", sort="-x", title="Modello LLM"),
        yOffset="machine_gpu:N",
        x=alt.X("tok_med_s:Q", title="Token al secondo"),
        color=alt.Color("machine_label:N"),  # leggenda già sopra
        opacity=alt.Opacity("gpu:N",
                            scale=opacity_scale, legend=None),
        strokeWidth=alt.condition(alt.datum.reference, alt.value(.5), alt.value(3)),
        tooltip=[
            "CPU", "GPU", "machine_gpu", "model", "gpu",
            alt.Tooltip("tok_med_s:Q", title="token/sec")
        ]
    )
)

text_oll = bars_oll.mark_text(align="left", dx=6).encode(
    text=alt.Text("tok_med_s:Q", format=".2f")
)

chart_oll = (bars_oll + text_oll).properties(
    width=650,
    title="Token al secondo – CPU vs GPU, per macchina"
).add_params(
    cpu_param, gpu_param, mach_param, mdl_param,
    gpu_tf_param, mach_legend_sel
)


In [24]:
chart_xgb

alt.LayerChart(...)

In [25]:
chart_oll

alt.LayerChart(...)